# This script is used to extract one certain gene’s information.

In [1]:
import glob
import json
import ast
import os

In [2]:
#define dir

# input dir
json_path = '../Autism_genepheno_results/Extraced_results'             # the output file of step1
np_dir = '../Autism_genepheno_results/Sum_all/n_p.txt'                # the output file of step1
ng_dir = '../Autism_genepheno_results/Sum_all/n_g.txt'                # the output file of step1
In_Summary_dir='../Autism_genepheno_results/Sum_all/In_Summary.txt'   # the output file of step1
NPMI_dir='../Autism_genepheno_results/NPMI_file/NPMI.json'            # the output file of step2

# output dir
one_information_dir = '../Autism_genepheno_results/one_gene_information/'

In [3]:
if os.path.exists(one_information_dir):
    print("using existing output folder: " + one_information_dir)
else:
    os.makedirs(one_information_dir)

using existing output folder: ../Autism_genepheno_results/one_gene_information/


In [38]:
# define the extracted gene
gene_extract = "SMS"

In [39]:
jsons = glob.glob("{}/*.json".format(json_path))

In [40]:
# define a dict to store the extracted gene information
gene_extract_property_dict = {"Gene name":gene_extract,
                              "Gene sfari class":None,
                              'Related sentences':{},
                              'Related phenotype NPMI':{},
                              'Summary':{"Paper number":None, 
                                         "Paper list":[],
                                         "Sentence number":None,
                                         "Normolized phenotype number":None}}

In [41]:
# read file in .\\Sum_all
def read_file():
    # read n_p
    with open(np_dir, 'r', encoding='utf-8-sig') as o:  
        phenotype_dict =  ast.literal_eval(o.read())
    
            
    # read n_g
    with open(ng_dir, 'r', encoding='utf-8-sig') as o:  
        gene_dict =  ast.literal_eval(o.read())
            
    # read n_tot
    with open(In_Summary_dir, 'r', encoding='utf-8-sig') as o:  
        f=o.read()
        h = f.find('N_tot = ')
        e = f.find('Unique gene list from all papers:')
        n_tot = int(f[h+8:e-2])
    return phenotype_dict, gene_dict, n_tot

phenotype_dict, gene_dict, n_tot = read_file()


In [42]:
# get gene_list and phenotype list from gene_dict and phenotype_dict
def get_list(gene_dict,phenotype_dict):
    gene_list=[]
    for key in gene_dict.keys():
        gene_list.append(key)
    phenotype_list=[]
    for key in phenotype_dict.keys():
        phenotype_list.append(key)
    return gene_list,phenotype_list

gene_list,phenotype_list = get_list(gene_dict,phenotype_dict)

In [43]:
# extract sentences which contain this gene from results in step 1
Sentence_number = 0
for index,item in enumerate(jsons) :
    with open(item, 'r', encoding='utf-8-sig') as f:
        json_data = json.load(f)

        for key in json_data['Sentences']:
            json_phenotype = json_data['Sentences'][key]['Normolized phenotype']
            json_gene = json_data['Sentences'][key]['Gene']
            if gene_extract in json_gene:
                Sentence_number += 1
                gene_extract_property_dict['Related sentences'].update({'Sentence'+str(Sentence_number).zfill(3):{'PMCid':json_data['PMCid'],'Title':json_data['Title']}})
                gene_extract_property_dict['Related sentences']['Sentence'+str(Sentence_number).zfill(3)].update(json_data['Sentences'][key])

In [44]:
# extract NPMI information from results in step 2
extracted_gene_NPMI_list = []

with open(NPMI_dir, 'r', encoding='utf-8-sig') as f:
    NPMI_data = json.load(f)

    for item in NPMI_data:
        if item['gene']==gene_extract:
            extracted_gene_NPMI_list.append(item)
    extracted_gene_NPMI_list.sort(key=lambda x: -x["NPMI"])

    
for index, item in enumerate(extracted_gene_NPMI_list):
    gene_extract_property_dict['Related phenotype NPMI'][item["phenotype"]] = item["NPMI"]
    if index ==0:
        gene_extract_property_dict["Gene sfari class"] = item["gene_sfari_class"]

In [45]:
# get summary of extracted gene information
gene_extract_property_dict['Summary']['Sentence number'] = Sentence_number
gene_extract_property_dict['Summary']['Normolized phenotype number'] = len(gene_extract_property_dict['Related phenotype NPMI'])
templist = []
templist2 = []
for sentence in gene_extract_property_dict['Related sentences']:
    templist.append(gene_extract_property_dict['Related sentences'][sentence]['PMCid'])
    templist2.append(gene_extract_property_dict['Related sentences'][sentence]['Title'])
    
gene_extract_property_dict['Summary']['Paper list'] = list(set(templist))
gene_extract_property_dict['Summary']['Paper name list'] = list(set(templist2))  
gene_extract_property_dict['Summary']['Paper number'] = len(gene_extract_property_dict['Summary']['Paper list'])

In [46]:
# write the json file
with open(one_information_dir+gene_extract+"_information.json", 'w', encoding='utf-8-sig') as f:
    json.dump(gene_extract_property_dict , f , sort_keys=True, indent=4, separators=(',', ': '))
                

In [47]:
# write the summary txt file
with open(one_information_dir+gene_extract+"_summary.txt", 'w', encoding='utf-8-sig') as f:
    f.write("Gene name: %s \n" % gene_extract)
    f.write("Paper number: %d \n" % gene_extract_property_dict['Summary']['Paper number']) 
    f.write("Paper list: \n" )
    for paper in gene_extract_property_dict['Summary']['Paper list'] :
        f.write(paper)
        f.write('\t')
    f.write('\n')        
    f.write("Paper name list: \n" )
    for paper in gene_extract_property_dict['Summary']['Paper name list'] :
        f.write(paper)
        f.write('\n')
    f.write('\n')
    f.write("Sentence number: %d \n" % gene_extract_property_dict['Summary']['Sentence number'])
    f.write("Normolized phenotype number: %d \n" % gene_extract_property_dict['Summary']['Normolized phenotype number'])    
    

In [37]:
len(gene_list)

6985

### Generating all the Gene reports

In [52]:
def generate_all_gene_reports(gene_list):
    for gene_extract in gene_list:
        
        print('Generating: ', gene_extract)
        
        jsons = glob.glob("{}/*.json".format(json_path))
        
        # define a dict to store the extracted gene information
        gene_extract_property_dict = {"Gene name":gene_extract,
                                      "Gene sfari class":None,
                                      'Related sentences':{},
                                      'Related phenotype NPMI':{},
                                      'Summary':{"Paper number":None, 
                                                 "Paper list":[],
                                                 "Sentence number":None,
                                                 "Normolized phenotype number":None}}
        
        # extract sentences which contain this gene from results in step 1
        Sentence_number = 0
        for index,item in enumerate(jsons) :
            with open(item, 'r', encoding='utf-8-sig') as f:
                json_data = json.load(f)

                for key in json_data['Sentences']:
                    json_phenotype = json_data['Sentences'][key]['Normolized phenotype']
                    json_gene = json_data['Sentences'][key]['Gene']
                    if gene_extract in json_gene:
                        Sentence_number += 1
                        gene_extract_property_dict['Related sentences'].update({'Sentence'+str(Sentence_number).zfill(3):{'PMCid':json_data['PMCid'],'Title':json_data['Title']}})
                        gene_extract_property_dict['Related sentences']['Sentence'+str(Sentence_number).zfill(3)].update(json_data['Sentences'][key])
                        
                        
        # extract NPMI information from results in step 2
        extracted_gene_NPMI_list = []
        with open(NPMI_dir, 'r', encoding='utf-8-sig') as f:
            NPMI_data = json.load(f)

            for item in NPMI_data:
                if item['gene']==gene_extract:
                    extracted_gene_NPMI_list.append(item)
            extracted_gene_NPMI_list.sort(key=lambda x: -x["NPMI"])


        for index, item in enumerate(extracted_gene_NPMI_list):
            gene_extract_property_dict['Related phenotype NPMI'][item["phenotype"]] = item["NPMI"]
            if index ==0:
                gene_extract_property_dict["Gene sfari class"] = item["gene_sfari_class"]                
        
        # get summary of extracted gene information
        gene_extract_property_dict['Summary']['Sentence number'] = Sentence_number
        gene_extract_property_dict['Summary']['Normolized phenotype number'] = len(gene_extract_property_dict['Related phenotype NPMI'])
        templist = []
        templist2 = []
        for sentence in gene_extract_property_dict['Related sentences']:
            templist.append(gene_extract_property_dict['Related sentences'][sentence]['PMCid'])
            templist2.append(gene_extract_property_dict['Related sentences'][sentence]['Title'])

        gene_extract_property_dict['Summary']['Paper list'] = list(set(templist))
        gene_extract_property_dict['Summary']['Paper name list'] = list(set(templist2))  
        gene_extract_property_dict['Summary']['Paper number'] = len(gene_extract_property_dict['Summary']['Paper list'])
        
        # write the json file
        with open(one_information_dir+gene_extract+"_information.json", 'w', encoding='utf-8-sig') as f:
            json.dump(gene_extract_property_dict , f , sort_keys=True, indent=4, separators=(',', ': '))
            
            # write the summary txt file
        with open(one_information_dir+gene_extract+"_summary.txt", 'w', encoding='utf-8-sig') as f:
            f.write("Gene name: %s \n" % gene_extract)
            f.write("Paper number: %d \n" % gene_extract_property_dict['Summary']['Paper number']) 
            f.write("Paper list: \n" )
            for paper in gene_extract_property_dict['Summary']['Paper list'] :
                f.write(paper)
                f.write('\t')
            f.write('\n')        
            f.write("Paper name list: \n" )
            for paper in gene_extract_property_dict['Summary']['Paper name list'] :
                f.write(paper)
                f.write('\n')
            f.write('\n')
            f.write("Sentence number: %d \n" % gene_extract_property_dict['Summary']['Sentence number'])
            f.write("Normolized phenotype number: %d \n" % gene_extract_property_dict['Summary']['Normolized phenotype number'])
            
            print('Finished: ', gene_extract)
    

In [53]:
generate_all_gene_reports(gene_list)

Generating:  ADNP
Finished:  ADNP
Generating:  TSC2
Finished:  TSC2
Generating:  TSC1
Finished:  TSC1
Generating:  TP53
Finished:  TP53
Generating:  T
Finished:  T
Generating:  COX7B
Finished:  COX7B
Generating:  HCCS
Finished:  HCCS
Generating:  NHS
Finished:  NHS
Generating:  ST8SIA2
Finished:  ST8SIA2
Generating:  MAF
Finished:  MAF
Generating:  GABRB3
Finished:  GABRB3
Generating:  GABRA3
Finished:  GABRA3
Generating:  GABRQ
Finished:  GABRQ
Generating:  MAOB
Finished:  MAOB
Generating:  FOXP2
Finished:  FOXP2
Generating:  PIAS1
Finished:  PIAS1
Generating:  FOXP1
Finished:  FOXP1
Generating:  FOXP4
Finished:  FOXP4
Generating:  TBR1
Finished:  TBR1
Generating:  SUMO2
Finished:  SUMO2
Generating:  SUMO1
Finished:  SUMO1
Generating:  SRPX2
Finished:  SRPX2
Generating:  FOXP3
Finished:  FOXP3
Generating:  NR5A1
Finished:  NR5A1
Generating:  SDS
Finished:  SDS
Generating:  F8
Finished:  F8
Generating:  TAT
Finished:  TAT
Generating:  HIBCH
Finished:  HIBCH
Generating:  ABO
Finished:  

Finished:  ERCC2
Generating:  SLC29A2
Finished:  SLC29A2
Generating:  ALDH3A1
Finished:  ALDH3A1
Generating:  IFNB1
Finished:  IFNB1
Generating:  NPPA
Finished:  NPPA
Generating:  DRD5
Finished:  DRD5
Generating:  CDA
Finished:  CDA
Generating:  NDUFB4
Finished:  NDUFB4
Generating:  DPYS
Finished:  DPYS
Generating:  SHMT1
Finished:  SHMT1
Generating:  KCNH2
Finished:  KCNH2
Generating:  CES1
Finished:  CES1
Generating:  LPL
Finished:  LPL
Generating:  DPYD
Finished:  DPYD
Generating:  SCNN1G
Finished:  SCNN1G
Generating:  PIK3CA
Finished:  PIK3CA
Generating:  SLC22A1
Finished:  SLC22A1
Generating:  UGT2B15
Finished:  UGT2B15
Generating:  FMR1
Finished:  FMR1
Generating:  CASP10
Finished:  CASP10
Generating:  IL10
Finished:  IL10
Generating:  CYP2B6
Finished:  CYP2B6
Generating:  CRHR2
Finished:  CRHR2
Generating:  MMP9
Finished:  MMP9
Generating:  CES2
Finished:  CES2
Generating:  TGFBR2
Finished:  TGFBR2
Generating:  XRCC3
Finished:  XRCC3
Generating:  ADRB2
Finished:  ADRB2
Generatin

Finished:  IFIT1
Generating:  CXCL12
Finished:  CXCL12
Generating:  ATG7
Finished:  ATG7
Generating:  ATG5
Finished:  ATG5
Generating:  CD48
Finished:  CD48
Generating:  BCR
Finished:  BCR
Generating:  VWF
Finished:  VWF
Generating:  IL1A
Finished:  IL1A
Generating:  FN1
Finished:  FN1
Generating:  CD86
Finished:  CD86
Generating:  IL17A
Finished:  IL17A
Generating:  PLG
Finished:  PLG
Generating:  KNG1
Finished:  KNG1
Generating:  STAB2
Finished:  STAB2
Generating:  GP6
Finished:  GP6
Generating:  SELPLG
Finished:  SELPLG
Generating:  MERTK
Finished:  MERTK
Generating:  AXL
Finished:  AXL
Generating:  BID
Finished:  BID
Generating:  U2AF1
Finished:  U2AF1
Generating:  SRSF2
Finished:  SRSF2
Generating:  SF3B1
Finished:  SF3B1
Generating:  CSF3R
Finished:  CSF3R
Generating:  TF
Finished:  TF
Generating:  PTS
Finished:  PTS
Generating:  CD5
Finished:  CD5
Generating:  CXCL2
Finished:  CXCL2
Generating:  GSDMA
Finished:  GSDMA
Generating:  B2M
Finished:  B2M
Generating:  HR
Finished:  HR

Finished:  C4B
Generating:  FUS
Finished:  FUS
Generating:  SOX10
Finished:  SOX10
Generating:  RET
Finished:  RET
Generating:  IL3RA
Finished:  IL3RA
Generating:  CSF2RA
Finished:  CSF2RA
Generating:  XIST
Finished:  XIST
Generating:  MALAT1
Finished:  MALAT1
Generating:  H19
Finished:  H19
Generating:  BDNF-AS
Finished:  BDNF-AS
Generating:  HTT
Finished:  HTT
Generating:  KCNQ1OT1
Finished:  KCNQ1OT1
Generating:  ATXN8OS
Finished:  ATXN8OS
Generating:  KCNQ1
Finished:  KCNQ1
Generating:  MBD1
Finished:  MBD1
Generating:  HMGB2
Finished:  HMGB2
Generating:  SORL1
Finished:  SORL1
Generating:  GABBR2
Finished:  GABBR2
Generating:  ATXN7
Finished:  ATXN7
Generating:  SMN2
Finished:  SMN2
Generating:  EZH1
Finished:  EZH1
Generating:  MED12
Finished:  MED12
Generating:  AKT3
Finished:  AKT3
Generating:  PTCH2
Finished:  PTCH2
Generating:  SOX4
Finished:  SOX4
Generating:  DENND1A
Finished:  DENND1A
Generating:  CS
Finished:  CS
Generating:  TREM2
Finished:  TREM2
Generating:  CASK
Finis

Finished:  WWC1
Generating:  NIPBL
Finished:  NIPBL
Generating:  UBC
Finished:  UBC
Generating:  SPRY2
Finished:  SPRY2
Generating:  MARK2
Finished:  MARK2
Generating:  CNTN6
Finished:  CNTN6
Generating:  ZNF423
Finished:  ZNF423
Generating:  RPS26
Finished:  RPS26
Generating:  CD14
Finished:  CD14
Generating:  ITGB1
Finished:  ITGB1
Generating:  RAB11FIP5
Finished:  RAB11FIP5
Generating:  SYT11
Finished:  SYT11
Generating:  RAP2A
Finished:  RAP2A
Generating:  GZMB
Finished:  GZMB
Generating:  PRF1
Finished:  PRF1
Generating:  TNFSF10
Finished:  TNFSF10
Generating:  CD6
Finished:  CD6
Generating:  IL18
Finished:  IL18
Generating:  CD38
Finished:  CD38
Generating:  CCL5
Finished:  CCL5
Generating:  CD3E
Finished:  CD3E
Generating:  LSR
Finished:  LSR
Generating:  OPRM1
Finished:  OPRM1
Generating:  GRIN2B
Finished:  GRIN2B
Generating:  MOG
Finished:  MOG
Generating:  HTR1B
Finished:  HTR1B
Generating:  DRD4
Finished:  DRD4
Generating:  HTR2C
Finished:  HTR2C
Generating:  TPH2
Finished: 

...

Finished:  MAGED2
Generating:  FIBIN
Finished:  FIBIN
Generating:  RNF40
Finished:  RNF40
Generating:  FOXK1
Finished:  FOXK1
Generating:  HOXD13
Finished:  HOXD13
Generating:  CRIPT
Finished:  CRIPT
Generating:  PTPN3
Finished:  PTPN3
Generating:  PDLIM2
Finished:  PDLIM2
Generating:  MNDA
Finished:  MNDA
Generating:  CENPE
Finished:  CENPE
Generating:  SASS6
Finished:  SASS6
Generating:  HMBOX1
Finished:  HMBOX1
Generating:  NMB
Finished:  NMB
Generating:  ZNF469
Finished:  ZNF469
Generating:  USH1C
Finished:  USH1C
Generating:  GOT1
Finished:  GOT1
Generating:  PAOX
Finished:  PAOX
Generating:  SPRN
Finished:  SPRN
Generating:  MTG1
Finished:  MTG1
Generating:  GPATCH8
Finished:  GPATCH8
Generating:  COPS3
Finished:  COPS3
Generating:  COL27A1
Finished:  COL27A1
Generating:  ERGIC1
Finished:  ERGIC1
Generating:  SPTBN4
Finished:  SPTBN4
Generating:  MBD6
Finished:  MBD6
Generating:  MED4
Finished:  MED4
Generating:  FBLIM1
Finished:  FBLIM1
Generating:  FASLG
Finished:  FASLG
Genera

Finished:  DYNLL1
Generating:  IMP3
Finished:  IMP3
Generating:  RCL1
Finished:  RCL1
Generating:  RIOK1
Finished:  RIOK1
Generating:  GRWD1
Finished:  GRWD1
Generating:  HEATR3
Finished:  HEATR3
Generating:  BCCIP
Finished:  BCCIP
Generating:  DEXI
Finished:  DEXI
Generating:  BCORL1
Finished:  BCORL1
Generating:  TNFRSF14
Finished:  TNFRSF14
Generating:  HSD3B1
Finished:  HSD3B1
Generating:  HADHB
Finished:  HADHB
Generating:  CSMD3
Finished:  CSMD3
Generating:  ZNF34
Finished:  ZNF34
Generating:  C1QTNF3
Finished:  C1QTNF3
Generating:  OLFM4
Finished:  OLFM4
Generating:  LMO1
Finished:  LMO1
Generating:  ZNF778
Finished:  ZNF778
Generating:  TRAPPC2L
Finished:  TRAPPC2L
Generating:  SMOC1
Finished:  SMOC1
Generating:  OTUD4
Finished:  OTUD4
Generating:  USP25
Finished:  USP25
Generating:  USP22
Finished:  USP22
Generating:  UCHL3
Finished:  UCHL3
Generating:  SSX3
Finished:  SSX3
Generating:  SSX9
Finished:  SSX9
Generating:  ZFAT
Finished:  ZFAT
Generating:  PSPC1
Finished:  PSPC1


Finished:  MSANTD2
Generating:  ORC4
Finished:  ORC4
Generating:  SHD
Finished:  SHD
Generating:  TRAF5
Finished:  TRAF5
Generating:  NAPEPLD
Finished:  NAPEPLD
Generating:  CEP131
Finished:  CEP131
Generating:  FOXN3
Finished:  FOXN3
Generating:  PRRX2
Finished:  PRRX2
Generating:  CDC42EP4
Finished:  CDC42EP4
Generating:  GPKOW
Finished:  GPKOW
Generating:  TRIM62
Finished:  TRIM62
Generating:  TRIM39-RPP21
Finished:  TRIM39-RPP21
Generating:  MRPL42
Finished:  MRPL42
Generating:  BOD1
Finished:  BOD1
Generating:  MIR548N
Finished:  MIR548N
Generating:  PRKRA
Finished:  PRKRA
Generating:  FBXO38
Finished:  FBXO38
Generating:  TGM7
Finished:  TGM7
Generating:  MICAL1
Finished:  MICAL1
Generating:  AKR1B15
Finished:  AKR1B15
Generating:  AKR1B10
Finished:  AKR1B10
Generating:  SGTA
Finished:  SGTA
Generating:  PASD1
Finished:  PASD1
Generating:  PPP1R15B
Finished:  PPP1R15B
Generating:  OSGEP
Finished:  OSGEP
Generating:  SEC31A
Finished:  SEC31A
Generating:  ARF4
Finished:  ARF4
Gener

Finished:  VTN
Generating:  PDK3
Finished:  PDK3
Generating:  HSPA13
Finished:  HSPA13
Generating:  POU2F2
Finished:  POU2F2
Generating:  KCNAB3
Finished:  KCNAB3
Generating:  TRAF3IP1
Finished:  TRAF3IP1
Generating:  MADCAM1
Finished:  MADCAM1
Generating:  ITGB7
Finished:  ITGB7
Generating:  VSX1
Finished:  VSX1
Generating:  MAGEC3
Finished:  MAGEC3
Generating:  CWC15
Finished:  CWC15
Generating:  BCO2
Finished:  BCO2
Generating:  GCHFR
Finished:  GCHFR
Generating:  PCSK2
Finished:  PCSK2
Generating:  BATF3
Finished:  BATF3
Generating:  ZNF300
Finished:  ZNF300
Generating:  FIGN
Finished:  FIGN
Generating:  PDLIM4
Finished:  PDLIM4
Generating:  WDFY2
Finished:  WDFY2
Generating:  PCDHGA9
Finished:  PCDHGA9
Generating:  PCDHB9
Finished:  PCDHB9
Generating:  NFATC4
Finished:  NFATC4
Generating:  RIN1
Finished:  RIN1
Generating:  STK32C
Finished:  STK32C
Generating:  LRP2BP
Finished:  LRP2BP
Generating:  ZNF148
Finished:  ZNF148
Generating:  SHISA6
Finished:  SHISA6
Generating:  SNORA10


Finished:  CST7
Generating:  LINC01476
Finished:  LINC01476
Generating:  AP2S1
Finished:  AP2S1
Generating:  STON2
Finished:  STON2
Generating:  AGFG1
Finished:  AGFG1
Generating:  SRPX
Finished:  SRPX
Generating:  ARMCX1
Finished:  ARMCX1
Generating:  TP53TG1
Finished:  TP53TG1
Generating:  PEAR1
Finished:  PEAR1
Generating:  SPHKAP
Finished:  SPHKAP
Generating:  LRRCC1
Finished:  LRRCC1
Generating:  IRX4
Finished:  IRX4
Generating:  IRX1
Finished:  IRX1
Generating:  SIGLEC10
Finished:  SIGLEC10
Generating:  ORAOV1
Finished:  ORAOV1
Generating:  CMTR1
Finished:  CMTR1
Generating:  ITGA7
Finished:  ITGA7
Generating:  HOXC13
Finished:  HOXC13
Generating:  GRB14
Finished:  GRB14
Generating:  STAG3L4
Finished:  STAG3L4
Generating:  MOV10L1
Finished:  MOV10L1
Generating:  BCAS1
Finished:  BCAS1
Generating:  SDC1
Finished:  SDC1
Generating:  PCSK7
Finished:  PCSK7
Generating:  CHST15
Finished:  CHST15
Generating:  IL12RB2
Finished:  IL12RB2
Generating:  IFNL3
Finished:  IFNL3
Generating:  I